<h1>Proyecto Tecnologías de Búsqueda en la Web</h1>
<h3>Integrantes</h3>
<ul><li>Sebastián Aranda</li><li>Felipe Santander</li></ul>

<h2>Declaraciones</h2>

In [5]:
import gensim
from gensim import corpora
from gensim.corpora import Dictionary, bleicorpus
from gensim.models import ldamodel
import pyLDAvis
from collections import defaultdict
import nltk
from nltk.corpus import stopwords
import os

In [6]:
corpus_path = "cyberbullying_corpus/"
#corpus_path = "corpus_lda/corpus_lda.lda_c"
#dictionary_path ="corpus_lda/corpus_lda.dict"

In [7]:
#Se eliminan palabras clave en deteccion de cyberbullying de la lista de stopwords de nltk
keywords = ["you", "your", "he", "she", "it"]
stopword_list = [stopword for stopword in stopwords.words('english') if stopword not in keywords]
puncts = ".,:;?!()[]{}~+-\"\'#$%&\/"
digits = "0123456789"

In [8]:
corpus = []
for filename in os.listdir(corpus_path):
    with open(corpus_path+filename) as file:
        tokens = []
        for line in file:
            for sym in puncts:
                line = line.replace(sym," ")
            for num in digits:
                line = line.replace(num," ")

            tokens = tokens + [word for word in line.lower().split() if word not in stopword_list]
        corpus.append(tokens)

# remove words that appear only once
frequency = defaultdict(int)
for text in corpus:
    for token in text:
        frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1] for text in corpus]

print(texts[:10])

[['first', 'time', 'you', 'held', 'cardboard', 'sign', 'you', 'least', 'impressive', 'ever', 'seen', 'lannister', 'always', 'never', 'payday', 'weird', 'three', 'creature', 'you', 'trying', 'fingerblast', 'shocker', 'you', 'look', 'like', 'version', 'joel', 'really', 'enjoyed', 'work', 'tucker', 'dale', 'vs', 'evil', 'http', 'www', 'imdb', 'com', 'media', 'rm', 'brain', 'cells', 'already', 'roasted', 'sure', 'see', 'phone', 'watch', 'back', 'side', 'sign', 'it', 'says', 'last', 'quarter', 'please', 'donate', 'your', 'life', 'would', 'want', 'high'], ['dude', 'drop', 'clothes', 'corner', 'your', 'boyfriend', 'waiting', 'you', 'room', 'intrigued', 'know', 'bears', 'prefer', 'tops', 'bottoms', 'grew', 'beard', 'hide', 'your', 'double', 'chins', 'like', 'use', 'word', 'intrigued', 'lot', 'heard', 'your', 'beard', 'shit', 'trapped', 'it', 'bathroom', 'wall', 'behind', 'you', 'you', 'know', 'gays', 'mustaches', 'beards', 'hide', 'stretch', 'marks'], ['comment', 'overwritten', 'open', 'source

<h2>Creación del Corpus</h2>

In [6]:
%time corpus = bleicorpus.BleiCorpus(corpus_path)
print(corpus)

%time dictionary = Dictionary.load(dictionary_path)
print(dictionary)

CPU times: user 68 ms, sys: 4 ms, total: 72 ms
Wall time: 92.4 ms
CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 8.11 ms
Dictionary(10734 unique tokens: [u'holyrood', u'four', u'francesco', u'cyprus', u'payoff']...)


In [7]:
%time lda_2 = ldamodel.LdaModel(corpus, num_topics=2, id2word = dictionary)
print(lda_2)

CPU times: user 26.4 s, sys: 20 ms, total: 26.4 s
Wall time: 27.1 s
LdaModel(num_terms=10734, num_topics=2, decay=0.5, chunksize=2000)
